In [ ]:
from langchain_ibm import WatsonxLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(template=
"""من فضلك صحّح الأخطاء الإملائية والنحوية في النصوص العربية التالية. ركّز على الأخطاء في التوافق بين الجنسين، التوافق في العدد، تصريف الأفعال، استخدام حروف الجر، الإملاء وعلامات التشكيل، وأخطاء التعريف. إليك تفصيل لأنواع الأخطاء مع أمثلة توضيحية:

صيغة الإخراج المطلوبة:
```json{{
    "error_type": "نوع الخطأ إذا وجد. أو (لا يوجد) إذا لم يوجد خطأ",
    "original_sentence": "الجملة الأصلية" ,
    "corrected_sentence": "الجملة بعد التصحيح"
    "modifications":{{
        "الكلمة الخطأ": [تصحيحها السليم]
    }}
}}```

أمثلة :

```json{{
        "error_type": "خطأ في التوافق بين الجنسين",
        "original_sentence": "الطفل جميلة ويحب اللعب في الحديقة.",
        "corrected_sentence": "الطفل جميل ويحب اللعب في الحديقة.",
        "modifications": {{
            "جميلة": "جميل"
}}
}}```,
```json{{
        "error_type": "خطأ في التوافق في العدد + خطأ في الإملاء (التاء المربوطة بدلا من الهاء)",
        "original_sentence": "المعلمتين يشرحون الدرس بطريقه ممتازة.",
        "corrected_sentence": "المعلمتان تشرحان الدرس بطريقة ممتازة.",
        "modifications": {{
            "المعلمتين": "المعلمتان",
            "يشرحون": "تشرحان",
            "بطريقه": "بطريقة"
        }}
    }}```,
```json{{
        "error_type": "خطأ في استخدام حروف الجر",
        "original_sentence": "هذه القصيده تحتوي على كثيراً من المعلومات.",
        "corrected_sentence": "هذه القصيدة تحتوي على الكثير من المعلومات.",
        "modifications": {{
            "القصيده": "القصيدة",
            "كثيراً من": "الكثير من"
}}
}}```,
```json{{
        "error_type": "خطأ في الإملاء وعلامات التشكيل",
        "original_sentence": "الطائره تقلع الساعه التاسعه.",
        "corrected_sentence": "الطائرة تقلع الساعة التاسعة.",
        "modifications": {{
            "الطائره": "الطائرة",
            "الساعه": "الساعة",
            "التاسعه": "التاسعة"
}}
}}```,

```json{{
"error_type": "خطأ في التوافق بين الجنسين + خطأ في التشكيل + خطأ في التذكير والتأنيث"
"original_sentence": "الطبيبه مهتم جداً بمرضى وتعاملهم بلطف شديد.",
"corrected_sentence": "الطبيبة مهتمة جداً بمرضاها وتعاملهم بلطف شديد",
"modifications": {{
    "الطبيبه": "الطبيبة",
    "مهتم": "مهتمة",
    "مرضى": "مرضاها"}}
}}```
,
```json{{
"error_type": "خطأ في تصريف الأفعال + خطأ في استخدام حرف الجر + خطأ في التذكير والتأنيث",
"original_sentence": "المهندسات كان لديهم خطط ممتاز لتنفيذ المشروع لكن لم ينفذوه بسبب نقص في المعدات.",
"corrected_sentence": "المهندسات كانت لديهن خطط ممتازة لتنفيذ المشروع لكن لم ينفذنه بسبب نقص المعدات"
"modifications": {{
    "كان": "كانت",
    "لديهم": "لديهن",
    "خطط ممتاز": "خطط ممتازة",
    "لم ينفذوه": "لم ينفذنه",
    "نقص في المعدات": "نقص المعدات"}}
}}```
,
```json{{
"error_type": "خطأ في التذكير والتأنيث + خطأ في التوافق في العدد + خطأ في التشكيل",
"original_sentence": "السيارات القديمه يحتاج إلى صيانه دوريه لأن محركهم مستهلكة.",
"corrected_sentence": "السيارات القديمة تحتاج إلى صيانة دورية لأن محركاتها مستَهلَكة"
"modifications": {{
    "القديمه": "القديمة",
    "يحتاج": "تحتاج",
    "صيانه": "صيانة",
    "دوريه" : "دورية"
    "محركهم": "محركاتها",
    "مستهلكة" : "مستَهلكة"
}}
}}```

قم بدورك الآن.

هذه هى الجملة المعطاة:
{sentence}

""", input_variables=['sentence'])


model_params = {
        "max_new_tokens": 1536,
        "min_new_tokens": 0,
        "temperature": 0.00,
}

watsonx_llm = WatsonxLLM(
    project_id="89b6a9d9-cb31-48fd-b5a4-9ed49fdaab52",
    apikey="tBmyiiTXb1TYJQPrYHOCjiek8iIQGZoqqZreZwrpSRCM",
    model_id="sdaia/allam-1-13b-instruct",
    url="https://eu-de.ml.cloud.ibm.com",
    params=model_params,
)

chain = prompt | watsonx_llm | JsonOutputParser()
query = "الشبابه كانو يمارسون اللعبه في الحدائق و يقدمو المساعده إلى الاطفال المحتاجين بفرح كبير و بلامبالاه."
res = chain.invoke({"sentence":query})
res

{'error_type': 'خطأ في التوافق بين الجنسين + خطأ في التوافق في العدد + خطأ في تصريف الأفعال + خطأ في استخدام حروف الجر + خطأ في الإملاء وعلامات التشكيل',
 'original_sentence': 'الشبابه كانو يمارسون اللعبه في الحدائق و يقدمو المساعده إلى الاطفال المحتاجين بفرح كبير و بلامبالاه.',
 'corrected_sentence': 'الشباب كانوا يمارسون اللعبة في الحدائق ويقدمون المساعدة إلى الأطفال المحتاجين بفرح كبير وبلا مبالاة.',
 'modifications': {'الشبابه': 'الشباب',
  'كانو': 'كانوا',
  'يمارسون': 'يمارسون',
  'العبه': 'اللعبة',
  'في الحدائق': 'في الحدائق',
  'يقدمو': 'يقدمون',
  'المساعده': 'المساعدة',
  'إلى الاطفال': 'إلى الأطفال',
  'المحتاجين': 'المحتاجين',
  'بفرح كبير': 'بفرح كبير',
  'بلامبالاه': 'بلا مبالاة'}}